In [7]:
import json
import tweepy
from kafka import KafkaProducer

# Twitter API authentication
bearer_token = "AAAAAAAAAAAAAAAAAAAAAITE9gAAAAAAehDsjjcy1lnlmuTrtNKwtz3erz4%3DJEVvSrVbVApD0zo3xpR2PAimOG13A56gFRpv3zHxOFFCJdTbCc"
client = tweepy.Client(bearer_token=bearer_token)

# Kafka producer setup
producer = KafkaProducer(
    bootstrap_servers="localhost:9092",
    value_serializer=lambda v: json.dumps(v).encode("utf-8")
)

topic_name = "tweet-crawl"

# Function to fetch tweets
def fetch_and_produce(query="AI", max_results=10):
    response = client.search_recent_tweets(query=query, tweet_fields=["id", "text"], max_results=max_results)
    if response.data:
        for tweet in response.data:
            tweet_data = {
                "id": tweet.id,
                "text": tweet.text
            }
            producer.send(topic_name, tweet_data)
            print(f"Produced -> {tweet_data}")

# Run producer
if __name__ == "__main__":
    fetch_and_produce()
    producer.flush()

NoBrokersAvailable: NoBrokersAvailable

In [9]:
import json
import psycopg2
from kafka import KafkaConsumer

# Kafka consumer setup
consumer = KafkaConsumer(
    "tweet-crawl",
    bootstrap_servers="localhost:9092",
    auto_offset_reset="earliest",
    enable_auto_commit=True,
    group_id="tweet-consumer-001",
     value_deserializer=lambda v: json.loads(v.decode("utf-8"))
)

# PostgreSQL connection
conn = psycopg2.connect(
    dbname="twitter_db",
    user="admin",
    password="admin123",
    host="localhost",
    port="5545"
)
cursor = conn.cursor()

# Create table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS tweet_merge (
    id BIGINT PRIMARY KEY,
    text TEXT
);
""")
conn.commit()

# Consume and insert into DB
for message in consumer:
    tweet = message.value
    try:
        cursor.execute(
            "INSERT INTO tweet_merge (id, text) VALUES (%s, %s) ON CONFLICT (id) DO NOTHING;",
            (tweet["id"], tweet["text"])
        )
        conn.commit()
        print(f"Inserted -> {tweet}")
    except Exception as e:
        print(f"Error inserting: {e}")
        conn.rollback()

NoBrokersAvailable: NoBrokersAvailable

In [ ]:
from pyspark.sql.functions import col, upper

# Convert tweet text column to uppercase
tweets_df = tweets_df.withColumn(
    "text_uppercase",
    upper(col("text"))
)

In [ ]:
# tweet_transform.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_extract, explode, split

# PostgreSQL connection details
jdbc_url = "jdbc:postgresql://host.docker.internal:5545/twitter_db"
db_properties = {
    "user": "admin",
    "password": "admin123",
    "driver": "org.postgresql.Driver"
}

# Create Spark session
spark = SparkSession.builder \
    .appName("TweetTransformation") \
    .getOrCreate()

# Read tweets from tweet_merge table
tweets_df = spark.read.jdbc(
    url=jdbc_url,
    table="tweet_merge",
    properties=db_properties
)

# Basic transformation: converts tweets to upeercase from tweet text
tweets_df = tweets_df.withColumn(
    "text_uppercase",
    upper(col("text"))
)

# Show sample transformed data
tweets_df.show(truncate=False)

# Write transformed data back to PostgreSQL (new table)
tweets_df.write.jdbc(
    url=jdbc_url,
    table="tweet_transformed",
    mode="overwrite",
    properties=db_properties
)

spark.stop()